From [openai_api_site](https://platform.openai.com/docs/guides/embeddings/use-cases)
Use cases
Here we show some representative use cases. We will use the Amazon fine-food reviews dataset for the following examples.

Obtaining the embeddings
The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text).

Codes which get the results **output/embedded_1k_reviews.csv**

In [20]:
# load & inspect dataset
input_datapath = "../data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...


In [21]:
df.size

7000

In [22]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

OSError: Cannot save file into a non-existent directory: 'output'

In [24]:
df.to_csv('output/embedded_1k_reviews.csv', index=False)

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('output/embedded_1k_reviews.csv')
df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)

In [2]:
df[:2]

,Time,ProductId,UserId,Score,Summary,Text,combined,ada_embedding
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[0.007060592994093895, -0.02732112631201744, 0..."
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,"[-0.023609420284628868, -0.011784634552896023,..."


In [3]:
len(df.iloc[0]["ada_embedding"])

1536

In [33]:
for i in range(5):
    print(df.iloc[i]['ada_embedding'])
    print(len(df.iloc[i]['ada_embedding']))

[ 0.00706059 -0.02732113  0.01058058 ... -0.00700559 -0.02187615
 -0.03756483]
1536
[-0.02360942 -0.01178463  0.00043665 ... -0.01466889  0.00732106
 -0.03327269]
1536
[ 0.00016697  0.00522649  0.00255825 ...  0.01033965 -0.00603646
 -0.01219959]
1536
[ 0.01053296 -0.01354704  0.0040275  ... -0.01375626 -0.03470449
 -0.02827095]
1536
[ 0.01525578 -0.00389863  0.01734025 ... -0.03019888 -0.00639023
 -0.02858341]
1536


In [32]:
for i in range(5):
    print(df.iloc[i]['combined'])
    print(len(df.iloc[i]['combined']))

Title: where does one  start...and stop... with a treat like this; Content: Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone
235
Title: Arrived in pieces; Content: Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.
142
Title: It isn't blanc mange, but isn't bad . . .; Content: I'm not sure that custard is really custard without eggs.  But this comes close.  I got it for use in a "Vegan pancake" recipe.  We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here.  One of the cooking/recipe sites had a recipe using this and there were lots of great reviews.  I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the  so-called custard isn't so bad.  I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor).  It's fun p